In [ ]:
%reload_ext autoreload
%autoreload 2

# Lab

# Entity

In [ ]:
import numpy as np
from munch import Munch


class Entity:
    def __init__(self, w, h, p=(0, 0), theta=0, name=None, color=None):
        self.bbox_float = np.array([[0, 0], [0, h], [w, h], [w, 0]])
        self.rotate(theta)
        self.translate(p)
        self.name = name
        self.color = color

    def draw(self, base, orientation_marker=True):
        d = ImageDraw.Draw(base)
        d.polygon(
            [tuple(p) for p in self.bbox],
            fill=self.color if self.color else (0, 0, 0, 255),
        )
        if orientation_marker:
            bottom_left = ((self.bbox_float[0] - self.bbox_float[1]) / 10).astype(
                int
            ) + self.top_left
            bottom_right = ((self.bbox_float[3] - self.bbox_float[2]) / 10).astype(
                int
            ) + self.top_right
            d.polygon(
                [
                    tuple(p)
                    for p in (bottom_left, self.top_left, self.top_right, bottom_right)
                ],
                fill=(255, 255, 255, 255),
            )
        return base

    def rotate(self, theta):
        c, s = np.cos(theta), np.sin(theta)
        R = np.array([[c, -s], [s, c]])
        self.bbox_float = self.bbox_float @ R.transpose()

    def translate(self, p):
        self.bbox_float += np.array(p)

    @property
    def bbox(self):
        """Round the oringinal bbox coordinates."""
        return self.bbox_float.round().astype(int)

    @property
    def bottom_left(self):
        return self.bbox[0]

    @property
    def top_left(self):
        return self.bbox[1]

    @property
    def top_right(self):
        return self.bbox[2]

    @property
    def bottom_right(self):
        return self.bbox[3]

    def __repr__(self):
        return f"{self.name}: {', '.join(str(tuple(p)) for p in self.bbox)}"


# Tests
def test_entity():
    w = 2
    h = 3
    p = (1, 2)
    theta = np.pi / 2

    entity = Entity(w, h, p=p, theta=theta)

    assert np.allclose(entity.top_left, np.array([-2, 2]))
    assert np.allclose(entity.top_right, np.array([-2, 4]))
    assert np.allclose(entity.bottom_left, np.array([1, 2]))
    assert np.allclose(entity.bottom_right, np.array([1, 4]))

# Relations

In [ ]:
def point_left_of_directed_line(point, dline):
    p = point
    q = dline.point
    d = dline.vector
    d_rot = np.array([-d[1], d[0]])  # d rotated by 90 degree
    return (p - q) @ d_rot > 0


def point_right_of_directed_line(point, dline):
    p = point
    q = dline.point
    d = dline.vector
    d_rot = np.array([-d[1], d[0]])  # d rotated by 90 degree
    return (p - q) @ d_rot < 0


def left_of(entity1, entity2):
    dline = Munch(
        point=entity2.bottom_left, vector=entity2.top_left - entity2.bottom_left
    )
    return all(
        point_left_of_directed_line(p, dline)
        for p in (
            entity1.top_left,
            entity1.top_right,
            entity1.bottom_left,
            entity1.bottom_right,
        )
    )


def right_of(entity1, entity2):
    dline = Munch(
        point=entity2.bottom_right, vector=entity2.top_right - entity2.bottom_right
    )
    return all(
        point_right_of_directed_line(p, dline)
        for p in (
            entity1.top_left,
            entity1.top_right,
            entity1.bottom_left,
            entity1.bottom_right,
        )
    )


def above(entity1, entity2):
    dline = Munch(point=entity2.top_left, vector=entity2.top_right - entity2.top_left)
    return all(
        point_left_of_directed_line(p, dline)
        for p in (
            entity1.top_left,
            entity1.top_right,
            entity1.bottom_left,
            entity1.bottom_right,
        )
    )


def below(entity1, entity2):
    dline = Munch(
        point=entity2.bottom_left, vector=entity2.bottom_right - entity2.bottom_left
    )
    return all(
        point_right_of_directed_line(p, dline)
        for p in (
            entity1.top_left,
            entity1.top_right,
            entity1.bottom_left,
            entity1.bottom_right,
        )
    )


# Tests
def test_left_of():
    entity1 = Entity(2, 3)
    entity2 = Entity(2, 3, p=(3, 3))
    assert left_of(entity1, entity2)


def test_right_of():
    entity1 = Entity(2, 3)
    entity2 = Entity(2, 3, p=(3, 3))
    assert right_of(entity2, entity1)

## Start with what you want to have and write tests.

In [ ]:
entity1 = Entity(32, 32, p=(32, 32), theta=30)
entity2 = Entity(32, 32, p=(64, 64), theta=30)

In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageOps

# make a blank image for the text, initialized to transparent text color
base = Image.new("RGBA", (224, 224), (255, 255, 255, 50))
entity1.draw(base)
ImageOps.flip(entity2.draw(base))

In [ ]:
base = Image.new("RGBA", (224, 224), (255, 255, 255, 50))
entity1.draw(base, orientation_marker=True)
ImageOps.flip(entity2.draw(base, orientation_marker=True))

In [ ]:
from ipywidgets import interact

@interact(
    x1=(0, 150),
    y1=(0, 150),
    w1=(10, 150),
    h1=(10, 150),
    theta1=(0.0, 2 * np.pi, 2 * np.pi / 360),
    x2=(0, 150),
    y2=(0, 150),
    w2=(10, 150),
    h2=(10, 150),
    theta2=(0.0, 2 * np.pi, 2 * np.pi / 360),
)
def test_spatial_relations(
    x1=3, y1=30, w1=30, h1=30, theta1=0.0, x2=60, y2=60, w2=30, h2=30, theta2=0.0
):
    base = Image.new("RGBA", (224, 224), (255, 255, 255, 20))
    entity1 = Entity(w1, h1, p=(x1, y1), theta=theta1, name="green", color="green")
    entity2 = Entity(w2, h2, p=(x2, y2), theta=theta2, name="red", color="red")
    entity1.draw(base)
    entity2.draw(base)
    display(ImageOps.flip(base))
    print(entity1)
    print(entity2)
    print(
        "left_of({}, {}): {}".format(
            entity1.name, entity2.name, left_of(entity1, entity2)
        )
    )
    print(
        "right_of({}, {}): {}".format(
            entity1.name, entity2.name, right_of(entity1, entity2)
        )
    )
    print(
        "above({}, {}): {}".format(entity1.name, entity2.name, above(entity1, entity2))
    )
    print(
        "below({}, {}): {}".format(entity1.name, entity2.name, below(entity1, entity2))
    )

In [ ]:
args = Munch(x1=3, y1=30, w1=30, h1=30, theta1=0.0, x2=60, y2=60, w2=30, h2=30, theta2=0.0)
base = Image.new("RGBA", (224, 224), (255, 255, 255, 20))
entity1 = Entity(args.w1, args.h1, p=(args.x1, args.y1), theta=args.theta1, name="green", color="green")
entity1.draw(base)

In [ ]:
np.array(entity1.draw(base))[:, :, 3].max()

In [ ]:
import math
import random
from itertools import product

from PIL import ImageOps

canvas_size = Munch(w=224, h=224)
num_entities = 5
ranges = Munch(w=(10, 30), h=(10, 30), x=(0, 224), y=(0, 224), theta=(0, math.pi))
relations = (left_of, right_of, above, below)


def inside_canvas(entity, canvas_size):
    xs_inside_canvas = all(
        (0 < entity.bbox[:, 0]) & (entity.bbox[:, 0] < canvas_size.w)
    )
    ys_inside_canvas = all(
        (0 < entity.bbox[:, 1]) & (entity.bbox[:, 1] < canvas_size.h)
    )
    return xs_inside_canvas and ys_inside_canvas


def in_relation(entity1, entity2, relations):
    return any(relation(entity1, entity2) for relation in relations)



entities_origin = []
for i in range(num_entities):
    w = random.uniform(*ranges.w)
    h = random.uniform(*ranges.h)
    entities_origin.append(Entity(w, h, name=str(i)))

In [ ]:
from copy import deepcopy

In [ ]:
entities_in_canvas = entities_in_relation = False
while not (entities_in_canvas and entities_in_relation):
    entities = deepcopy(entities_origin)
    for entity in entities:
        theta = random.uniform(*ranges.theta)
        entity.rotate(theta)

    for entity in entities:
        p = (random.uniform(*ranges.x), random.uniform(*ranges.y))
        entity.translate(p)

    entities_in_canvas = all(inside_canvas(entity, canvas_size) for entity in entities)
    entities_in_relation = all(
        in_relation(entity1, entity2, relations)
        for entity1, entity2 in product(entities, repeat=2)
        if entity1 != entity2
    )


# Draw the image vertically flipped
canvas = Image.new("RGBA", (canvas_size.h, canvas_size.w), (255, 255, 255, 20))
for entity in entities:
    entity.draw(canvas)
display(ImageOps.flip(canvas))

In [ ]:
entities

In [ ]:
all(
    in_relation(entity1, entity2, relations)
    for entity1, entity2 in product(entities, repeat=2)
    if entity1 != entity2
)

In [ ]:
canvas = Image.new("RGBA", (canvas_size.h, canvas_size.w), (255, 255, 255, 20))
ImageOps.flip(entities[0].draw(canvas))

In [ ]:
canvas = Image.new("RGBA", (canvas_size.h, canvas_size.w), (255, 255, 255, 20))
ImageOps.flip(entities[1].draw(canvas))

In [ ]:
in_relation(entities[0], entities[1], relations)

In [ ]:
left_of(entities[0], entities[1])